# Hugging Face Large Model Inference

This notebook demonstrates how to deploy Hugging Face transformer models using Hugging Face Text Generation Inference (TGI) Deep Learning Container on Amazon SageMaker.

TGI is an open source, high performance inference library that can be used to deploy large language models from Hugging Face’s repository in minutes. The library includes advanced functionality like model parallelism and dynamic batching to simplify production inference with large language models like flan-t5-xxl, LLaMa, StableLM, and GPT-NeoX. 

## Setup

### Install the SageMaker Python SDK

First, make sure that the latest version of SageMaker SDK is installed.

In [1]:
%pip install "sagemaker>=2.163.0"

Note: you may need to restart the kernel to use updated packages.


### Setup account and role

Then, we import the SageMaker python SDK and instantiate a `sagemaker_session` which we use to determine the current region and execution role.

In [2]:
import sagemaker
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
import time

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## Retrieve the LLM Image URI

We use the helper function `get_huggingface_llm_image_uri()` to generate the appropriate image URI for the Hugging Face Large Language Model (LLM) inference.

The function takes a required parameter `backend` and several optional parameters. The `backend` specifies the type of backend to use for the model, the values can be "lmi" and "huggingface". The "lmi" stands for SageMaker LMI inference backend, and "huggingface" refers to using Hugging Face TGI inference backend.

In [4]:
image_uri = get_huggingface_llm_image_uri(
  backend="huggingface", # or lmi
  region=region
)

## Create the Hugging Face Model

Next we configure the `model` object by specifying a unique name, the `image_uri` for the managed TGI container, and the execution role for the endpoint. Additionally, we specify a number of environment variables including the `HF_MODEL_ID` which corresponds to the model from the HuggingFace Hub that will be deployed, and the `HF_TASK` which configures the inference task to be performed by the model.

You should also define `SM_NUM_GPUS`, which specifies the tensor parallelism degree of the model. Tensor parallelism can be used to split the model across multiple GPUs, which is necessary when working with LLMs that are too big for a single GPU. Here, you should set `SM_NUM_GPUS` to the number of available GPUs on your selected instance type.  For example, in this tutorial, we set `SM_NUM_GPUS` to 4 because our selected instance type `ml.g4dn.12xlarge` has 4 available GPUs.

Additionally, we could reduce the memory footprint of the model by setting the `HF_MODEL_QUANTIZE` environment variable to true.

In [11]:
model_name = "Mistral-7B-Instruct-v2-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

hub = {
    'HF_MODEL_ID':'mistralai/Mistral-7B-Instruct-v0.2',
    'HF_TASK':'text-generation',
    'SM_NUM_GPUS':'1',
    'HF_MODEL_QUANTIZE': 'bitsandbytes-nf4',
}

model = HuggingFaceModel(
    name=model_name,
    env=hub,
    role=role,
    image_uri=image_uri
)

## Creating a SageMaker Endpoint

Next we deploy the model by invoking the `deploy()` function. Here we use an `ml.g4dn.12xlarge` instance which come with 4 NVIDIA A10 GPUs. By setting the `SM_NUM_GPUS` environment variable to 4 in the last code block, we indicate that this model should be sharded across all 4 GPU devices. 

In [12]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge",
    endpoint_name=model_name
)

-----------!

## Running Inference

Once the endpoint is up and running, we can evaluate the model using the `predict()` function.

In [18]:
%%time
input_data = {
  "inputs": "Write a blog about foundation model ops",
  "parameters": {
    "do_sample": True,
    "max_new_tokens": 2000,
    "temperature": 0.7,
    "watermark": True
  }
}

predictor.predict(input_data)

CPU times: user 3.91 ms, sys: 16 µs, total: 3.93 ms
Wall time: 17.3 s


[{'generated_text': "Write a blog about foundation model ops and how they are different from traditional machine learning pipelines.\n\nTitle: Foundation Model Operations: A Shift from Traditional Machine Learning Pipelines\n\nAs technology advances and artificial intelligence becomes an increasingly integral part of business processes, the way we approach machine learning models is evolving. One significant change is the emergence of foundation model operations, a new paradigm for managing machine learning workflows that deviates from the traditional machine learning pipelines. In this blog post, we will discuss what foundation model operations are, how they differ from traditional machine learning pipelines, and the implications of this shift.\n\nFirst, let's define foundation models and foundation model operations. Foundation models are large, general-purpose machine learning models that have been pre-trained on vast amounts of data and can perform various tasks with minimal fine-tu

In [19]:
2000/17.3

115.60693641618496

## Cleaning Up

After you've finished using the endpoint, it's important to delete it to avoid incurring unnecessary costs.

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

## Conclusion

In this tutorial, we used a TGI container to deploy GPT-NeoX using 4 GPUs on a SageMaker `ml.g4dn.12xlarge` instance. With Hugging Face's Text Generation Inference and SageMaker Hosting, you can easily host large language models like GPT-NeoX, flan-t5-xxl, and LLaMa.